In [1]:
import os 
from dotenv import load_dotenv
from langchain.memory import ConversationBufferWindowMemory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
load_dotenv()

True

In [4]:
from openai import OpenAI
import httpx
import os

# As mentioned in the above Note try to setup Dell certificate in your environment to avoid verify=False(SSL verification is disabled).
http_client=httpx.Client(verify=False)
client = OpenAI(
    base_url='https://opensource-challenger-api.prdlvgpu1.aiaccel.dell.com/v1',
    http_client=http_client,
    api_key=os.environ['LLM_API']
)
streaming = True
max_output_tokens = 100

# Read the content of 'output.txt' as the required prompt
# with open('../output.txt', 'r') as f:
#     required_prompt = f.read()

# Available Models list
available_models = ["mixtral-8x7b-instruct-v01", "llamaguard-7b", "gemma-7b-it", "mistral-7b-instruct-v02", "phi-2", "llama-2-70b-chat", "phi-3-mini-128k-instruct", "llama-3-8b-instruct"]

# Let's select the model from the available list
model_selected = available_models[0]


In [3]:
with open('../output1.txt', 'r') as f:
    required_prompt = f.read()
len(required_prompt)

56548

In [24]:


print(f"Model: {model_selected}")
completion = client.completions.create(
    model=model_selected,
    max_tokens=max_output_tokens,
    prompt="Hi, I am bob, tell me about yourself.",
    stream=streaming
)

if streaming:
    for chunk in completion:
        print(chunk.choices[0].text, end='')
else:
    print(completion.choices[0].text)


Model: mixtral-8x7b-instruct-v01

Hi Bob, I am a bot, I analyze input and help find ways to solve problems. How can I help you today?

Recently, there's been a seemingly endless number of generative AI projects, from "fine-tuned" public models to fully proprietary closed systems. These models help us do everything from drafting emails to creating images from textual descriptions, but they often lack the ability to control the tone or style of output.

I

In [12]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI


store = {}  # memory is maintained outside the chain

def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

llm = ChatOpenAI(
    base_url='https://opensource-challenger-api.prdlvgpu1.aiaccel.dell.com/v1',
    model=model_selected,
    openai_api_key=os.environ['LLM_API'],
    http_client=http_client
)

chain = RunnableWithMessageHistory(llm, get_session_history)
chain.invoke(
    "Hi I'm Bob.",
    config={"configurable": {"session_id": "1"}},
)  # session_id determines thread

Parent run 7f4a3b7a-c43e-4d4d-8d02-32a536c98eae not found for run b5e7127d-d482-4809-af11-4fa954e8ebde. Treating as a root run.


AIMessage(content=" Hello Bob! It's nice to meet you. How can I help you today? Is there something you would like to know or talk about? I'm here to answer any questions you have to the best of my ability.", response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 14, 'total_tokens': 62}, 'model_name': 'mixtral-8x7b-instruct-v01', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b5e7127d-d482-4809-af11-4fa954e8ebde-0', usage_metadata={'input_tokens': 14, 'output_tokens': 48, 'total_tokens': 62})

In [14]:



store = {}  # memory is maintained outside the chain

def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
        return store[session_id]

    memory = ConversationBufferWindowMemory(
        chat_memory=store[session_id],
        k=3,
        return_messages=True,
    )
    assert len(memory.memory_variables) == 1
    key = memory.memory_variables[0]
    messages = memory.load_memory_variables({})[key]
    store[session_id] = InMemoryChatMessageHistory(messages=messages)
    return store[session_id]

llm = ChatOpenAI(base_url='https://opensource-challenger-api.prdlvgpu1.aiaccel.dell.com/v1',
    model=model_selected,
    openai_api_key=os.environ['LLM_API'],
    http_client=http_client)

chain = RunnableWithMessageHistory(llm, get_session_history)
chain.invoke(
    "Hi I'm Bob.",
    config={"configurable": {"session_id": "1"}},
)  # session_id determines thread

Parent run 01dee4e9-428e-4a98-9585-bec7cf7ef13b not found for run fb4faafb-4435-4391-aa85-d8c13d9b8e3b. Treating as a root run.


AIMessage(content=" Hello Bob! It's nice to meet you. Is there something specific you would like to talk about or ask me? I'm here to help with any questions you have to the best of my ability.", response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 15, 'total_tokens': 59}, 'model_name': 'mixtral-8x7b-instruct-v01', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-fb4faafb-4435-4391-aa85-d8c13d9b8e3b-0', usage_metadata={'input_tokens': 15, 'output_tokens': 44, 'total_tokens': 59})

In [5]:
print(f"Model: {model_selected}")

# Chat history management
store = {}  # memory is maintained outside the chain

def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

def print_chat_history(session_id: str):
    history = get_session_history(session_id)
    print(f"\nChat History for session {session_id}:")
    for message in history.messages:
        print(f"{message.type}: {message.content}")

# Create a proper LLM object
llm = ChatOpenAI(
    base_url='https://opensource-challenger-api.prdlvgpu1.aiaccel.dell.com/v1',
    model=model_selected,
    openai_api_key=os.environ['LLM_API'],
    http_client=http_client
)

chain = RunnableWithMessageHistory(llm, get_session_history)

# Check initial state
print_chat_history("1")

# First interaction
print("\nFirst interaction:")
response1 = chain.invoke(
    "Hi, I'm Bob. What's your name?",
    config={"configurable": {"session_id": "1"}},
)
print("Assistant:", response1.content)

# Check history after first interaction
print_chat_history("1")

# Second interaction
print("\nSecond interaction:")
response2 = chain.invoke(
    "Nice to meet you! What do you know about Python?",
    config={"configurable": {"session_id": "1"}},
)
print("Assistant:", response2.content)

# Check history after second interaction
print_chat_history("1")

# Third interaction with a different session
print("\nThird interaction (different session):")
response3 = chain.invoke(
    "Hello, I'm Alice. Can you tell me about the weather?",
    config={"configurable": {"session_id": "2"}},
)
print("Assistant:", response3.content)

# Check history for both sessions
print_chat_history("1")
print_chat_history("2")

Model: mixtral-8x7b-instruct-v01


Parent run 18e5518c-27c9-43b1-a16d-2347b38ae88c not found for run 910a9025-6d3f-48eb-8df0-cdecb7af0cd8. Treating as a root run.



Chat History for session 1:

First interaction:


Parent run 82dc0031-c33e-4ca0-8e4d-cd2fe3dab462 not found for run f8978762-d9b5-4cdc-b2c0-9f3dff6b7a82. Treating as a root run.


Assistant:  Hello Bob! I'm an assistant. I don't have a personal name as I'm a computer program, but you can call me Assistant. How can I assist you today?

Chat History for session 1:
human: Hi, I'm Bob. What's your name?
ai:  Hello Bob! I'm an assistant. I don't have a personal name as I'm a computer program, but you can call me Assistant. How can I assist you today?

Second interaction:


Parent run df919f89-3c64-4309-8719-a5fd81489ff9 not found for run 9139a873-70f7-44d5-ae0c-46826d340109. Treating as a root run.


Assistant:  Hello Bob! Nice to meet you too. I can tell you that Python is a high-level, interpreted programming language that was created by Guido van Rossum and first released in 1991. It is designed with an emphasis on code readability, and its syntax allows programmers to express concepts in fewer lines of code than would be possible in languages such as C++ or Java.

Python supports multiple programming paradigms, including procedural, object-oriented, and functional programming. It is commonly used for web development, data analysis, artificial intelligence, machine learning, and scientific computing, among other applications.

Python has a large standard library that includes areas like internet protocols, string operations, web services tools, and operating system interfaces. Many high use programming tasks are already scripted into the standard library which reduces length of code to be written significantly.

Python is also known for its strong support for integration with ot

In [16]:
with open('../output1.txt', 'r') as f:
    required_prompt = f.readlines()

In [17]:
type(required_prompt)

list

In [18]:
required_prompt[:10]

["vram\\EmulatorPkg\\Unix\\lldbinit:imports-['command script import lldbefi.py']\n",
 'vram\\MdeModulePkg\\Library\\BrotliCustomDecompressLib\\brotli\\configure-cmake:imports-[\'#   You must include the GNUInstallDirs module (which comes with\', \'#   CMake) in your project.  Just put "include (GNUInstallDirs)" in\']\n',
 "vram\\MdeModulePkg\\Library\\BrotliCustomDecompressLib\\brotli\\Makefile:imports-['$(CC) $(CFLAGS) $(CPPFLAGS) -Ic/include \\\\']\n",
 'vram\\MdeModulePkg\\Library\\BrotliCustomDecompressLib\\brotli\\python\\Makefile:imports-[\'EXT_SUFFIX=$(shell $(PYTHON) -c \\\'import sysconfig; print(sysconfig.get_config_var("EXT_SUFFIX"))\\\')\']\n',
 'vram\\MdeModulePkg\\Universal\\RegularExpressionDxe\\oniguruma\\INSTALL:imports-[\'below.  The lack of an optional feature in a given package is not\', \'The simplest way to compile this package is:\', "\'./configure\' to configure the package for your system.", \'recommended that the package be configured and built as a regular\',

In [19]:
required_prompt = "".join(required_prompt[:30])

In [20]:
print(len(required_prompt))

8514


Feed this into a conversational chain

In [27]:
print(f"Model: {model_selected}")

# Chat history management
store = {}  # memory is maintained outside the chain

def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

def print_chat_history(session_id: str):
    history = get_session_history(session_id)
    print(f"\nChat History for session {session_id}:")
    for message in history.messages:
        print(f"{message.type}: {message.content}")

# Create a proper LLM object
llm = ChatOpenAI(
    base_url='https://opensource-challenger-api.prdlvgpu1.aiaccel.dell.com/v1',
    model=model_selected,
    openai_api_key=os.environ['LLM_API'],
    http_client=http_client
)

chain = RunnableWithMessageHistory(llm, get_session_history)

# Check initial state
print_chat_history("1")

# First interaction
print("\nFirst interaction:")
response1 = chain.invoke(
    required_prompt+"Understand the codebase and the project structure.",
    config={"configurable": {"session_id": "1"}},
)
print("Assistant:", response1.content)

# Check history after first interaction
print_chat_history("1")

# Second interaction
print("\nSecond interaction:")
response2 = chain.invoke(
    "Can you generate test automation scripts on this project given the information or do you need more?",
    config={"configurable": {"session_id": "1"}},
)
print("Assistant:", response2.content)

# Check history after second interaction
print_chat_history("1")



Model: mixtral-8x7b-instruct-v01


Parent run e1666d88-233b-492c-8be1-b742ab26a60d not found for run 5e888432-2367-470f-b70e-e89906f26dc0. Treating as a root run.



Chat History for session 1:

First interaction:


Parent run 04d9e985-0978-4fa0-8c85-937bcadd5eca not found for run 1abbd9e5-6275-4201-99c0-2485c9d0e699. Treating as a root run.


Assistant:  The text you provided appears to be fragments of various configuration and documentation files from a codebase. I will attempt to summarize the information contained in these fragments.

1. lldbinit: This file is related to the LLDB debugger and imports a Python module `lldbefi.py`.
2. configure-cmake: This file is a CMake configuration script for the BrotliCustomDecompressLib library and mentions the GNUInstallDirs module.
3. Makefile: This is a Makefile for the BrotliCustomDecompressLib library, using the C compiler and including headers from the `c/include` directory.
4. python/Makefile: This is a Makefile for building Python extensions in the BrotliCustomDecompressLib library, extracting the extension suffix from the Python configuration.
5. INSTALL: This is a generic installation instruction file for the oniguruma package, mentioning various ways to configure and compile the package for different systems and environments.
6. depcomp: This file is an Autoconf script for

It kinda works I think the prompts need to be more streamlined and then I can work on the post processing and preprocessing of the file

In [8]:
with open('../output1.txt', 'r') as f:
    content = f.readlines()

store = {}  # memory is maintained outside the chain

def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

llm = ChatOpenAI(
    base_url='https://opensource-challenger-api.prdlvgpu1.aiaccel.dell.com/v1',
    model=model_selected,
    openai_api_key=os.environ['LLM_API'],
    http_client=http_client
)

chain = RunnableWithMessageHistory(llm, get_session_history)


current_context = "".join(content[:10])
ctr = 10
llm_request = "need more"
while llm_request == "need more":
    print(f"Current Context: {len(current_context)}")
    response = chain.invoke(
        current_context+" write code for test automation of the given context or say 'need more' to request for more information.",
        config={"configurable": {"session_id": "1"}},
    )
    print("Assistant:", response.content)
    if 'need' in response.content.lower() or 'more' in response.content.lower():
        llm_request = "need more"
        ctr+=10
        current_context = "".join(content[ctr:ctr+10]) if ctr+10 < len(content) else "".join(content[ctr:])
    else:
        llm_request = "done"



Parent run 9fb9f5f6-28bc-4a45-91fb-6093a7525261 not found for run 54070895-1cce-4a90-85b9-74717ca62428. Treating as a root run.


Current Context: vram\MdePkg\Include\Library\BaseLib.h:imports-['that the 16-bit thunk functions require for']
vram\MdePkg\Include\Library\BaseRiscVSbiLib.h:imports-['#include <Uefi.h>']
vram\MdePkg\Include\Library\CpuLib.h:imports-['PAL Calls require PEI and DXE specific mechanisms to look up PAL Entry Point.']
he DXE Services', 'Table should use this library.  This will typically be DXE Drivers that require GCD']
vram\MdePkg\Include\Library\CcProbeLib.h:imports-['#include <ConfidentialComputingGuestAttr.h>']
vram\MdePkg\Include\Library\FileHandleLib.h:imports-['#include <Protocol/SimpleFileSystem.h>', '#include <Guid/FileInfo.h>']
vram\MdePkg\Include\Library\ExtractGuidedSectionLib.h:classes-['A library instance that produces this library class may be used to produce a']
vram\MdePkg\Include\Library\MmServicesTableLib.h:imports-['#include <PiMm.h>']
vram\MdePkg\Include\Library\PeCoffExtraActionLib.h:imports-['#include <Library/PeCoffLib.h>']
>']



Parent run 2354d80b-cc35-4867-b194-2f65695253f7 not found for run 1687d291-e863-4bd2-bbf3-b9fb89c5477f. Treating as a root run.


Assistant:  Based on the provided context, it seems like you are asking for a code snippet for test automation of the given UEFI libraries. However, the context does not provide sufficient information about the specific functionality or behavior that needs to be tested.

To provide a useful code snippet, I would need to know more about the following:

1. What is the specific functionality or behavior that needs to be tested?
2. Which library or libraries are involved in this functionality or behavior?
3. What are the input parameters and expected output or behavior for this functionality or behavior?

Once I have this information, I can provide a more targeted code snippet for test automation.

In general, test automation for UEFI libraries would involve writing test cases in a programming language such as C or C++, using a UEFI test framework such as UTK or EFIX devkit. The test cases would call the relevant library functions with different input parameters and verify that the output 

Parent run 0d1043b6-608f-4855-b9bf-38949d404383 not found for run c9805828-fff4-4a66-bf6f-0eb6b855c34c. Treating as a root run.


...

Assistant:  To write test automation code for the given UEFI libraries and protocols, we would need to create test cases that verify their behavior and functionality. Here is an example of how we might write a test case for the `PcdInfo` library:

```c
#include <Uefi.h>
#include <PcdInfo.h>

EFI_STATUS
EfiPcdInfoTest (
    VOID
    )
{
    EFI_STATUS                 Status;
    EFI_PEI_SERVICES          **PeiServices;
    EFI_PEI_PCD_INFO_PPI      *PcdInfoPpi;
    EFI_PEI_PCD_INFO          *PcdInfo;
    EFI_PEI_PCD_INFO_ITEM     *Item;
    UINTN                      ItemCount;

    // Get the PEI Services
    Status = PeiServicesLocatePpi (
               &gEfiPeiServicesGuid,
               NULL,
               (VOID **) &PeiServices
               );
    if (EFI_ERROR (Status)) {
        return Status;
    }

    // Get the PCD Info PPI
    Status = PeiServicesLocatePpi (
               &gEfiPeiPcdInfoPpiGuid,
               NULL,
               (VOID **) &PcdInfoPpi
               )

Parent run a7819493-d613-4024-8a80-09b2a383e969 not found for run ec27113d-3902-4b1f-97c4-163ced38c3d1. Treating as a root run.


Assistant:  To write test automation code for the given UEFI protocols, we would need to create test cases that verify their behavior and functionality. Here is an example of how we might write a test case for the `BluetoothLeConfig` protocol:

```c
#include <Uefi.h>
#include <BluetoothLeConfig.h>

EFI_STATUS
EfiBluetoothLeConfigTest (
    VOID
    )
{
    EFI_STATUS                 Status;
    EFI_BLUETOOTH_LE_CONFIG   *BluetoothLeConfig;
    EFI_GUID                   Guid = BLUETOOTH_LE_CONFIG_PROTOCOL_GUID;

    // Get the Bluetooth LE Config protocol
    Status = gBS->LocateProtocol (&Guid, NULL, (VOID **) &BluetoothLeConfig);
    if (EFI_ERROR (Status)) {
        return Status;
    }

    // Test the SetFeature function
    Status = BluetoothLeConfig->SetFeature (BluetoothLeConfig, &gEfiBluetoothLeFeatureDataLength, sizeof (gEfiBluetoothLeFeatureDataLength));
    if (EFI_ERROR (Status)) {
        return Status;
    }

    // Test the GetFeature function
    UINT8 DataLength;
    

Parent run 3ade55be-8b50-4ec0-a5ae-6545ba5dca33 not found for run 5b379ebe-ca2c-437f-a0ea-dc3e9b5309a7. Treating as a root run.


Assistant:  To write test automation code for the given UEFI protocols, we would need to create test cases that verify their behavior and functionality. Here is an example of how we might write a test case for the `DriverHealth` protocol:

```c
#include <Uefi.h>
#include <DriverHealth.h>

EFI_STATUS
EfiDriverHealthTest (
    VOID
    )
{
    EFI_STATUS                         Status;
    EFI_DRIVER_HEALTH_PROTOCOL       *DriverHealth;
    EFI_DRIVER_HEALTH_CONTROLLER_DATA ControllerData;

    // Get the Driver Health protocol
    Status = gBS->LocateProtocol (&gEfiDriverHealthProtocolGuid, NULL, (VOID **) &DriverHealth);
    if (EFI_ERROR (Status)) {
        return Status;
    }

    // Test the GetControllerHealth function
    Status = DriverHealth->GetControllerHealth (DriverHealth, &gEfiDriverHealthControllerGuid, &ControllerData);
    if (EFI_ERROR (Status)) {
        return Status;
    }

    // Check the health status of the controller
    if (ControllerData.HealthStatus == EfiDr